Machine Learning - Homework 1 (due Sep. 13)

Problem 1: Legal reasoning (from Murphy 2.2).

Suppose a crime has been committed. Blood is found at the scene for which there is no innocent explanation. It is of a type which is present in 1% of the population. The defendant has this blood type.

a. The prosecutor claims: “There is a 1% chance that the defendant would have the crime blood type if he
were innocent. Thus there is a 99% chance that he guilty”. This is known as the prosecutor’s fallacy.
What is wrong with this argument?

The procecutor is not taking into account and prior knowledge about whether or not the defendant is guilty. Bayes Theorem states:
$$ P(G|B) = \frac{P(B|G)P(G)}{P(B|G)P(G) + P(B|\neg G)P(\neg G)}$$ 
$$  = \frac{P(B|G)P(G)}{P(B|G)P(G) + 0.01(1-P(G))}$$
$$  = \frac{P(B|G)P(G)}{(P(B|G) -0.01)P(G) + 0.01}$$
If P(G) is high and P(B|G) is high it could be close to $99 \%$, but as each number decreases so does P(G|B).  

b. The defender claims: “The crime occurred in a city of 800,000 people. The blood type would be
found in approximately 8000 people. The evidence has provided a probability of just 1 in 8000 that
the defendant is guilty, and thus has no relevance.” This is known as the defender’s fallacy. What is
wrong with this argument?

This is not taking into account any prior probability that the defendant is guilty. If the blood type matches, the suspect pool is drastically reduced, increasing the chances that the defendant is guilty. Subing in $P(G) = 1/800000$ and $P(G) = 1/8000$ into the formula above gives:

$$P(G|B) = \frac{P(B|G)*1/800000}{P(B|G)*1/800000 + 1/100 * 799999/800000}$$

$$P(G|B) = \frac{100*P(B|G)}{100*P(B|G)+799999)}$$

Compared to:

$$P(G|B) = \frac{100*P(B|G)}{100*P(B|G)+7999)}$$

Since the numerators are the same, and the second denominator is much smaller, if the defendants blood type matches the probability of guilt goes up considerably.

c. Suppose that forensic analysis tells us that with 98% sensitivity (true positive rate) and a 1% false positive rate, the blood found at the scene belongs to the perpetrator.  Given the information presented in the above two questions, determine the posterior probability the the defendent is guilty, given that the defendent's blood type matches that found at the crime scene *and* that the defendent was one of only 5 people with access to the crime scene.  

In [5]:
x =.98*.2/(.98*0.2+.01*.8)
print(x)


0.9607843137254901



There would be a $96\%$ probability that the defendant is guilty.

Problem 2: Naive Bayes

One use of the naive Bayes classifier, which is still in practical use today, is as a spam filter.  Consider the corpus of text messages packaged with this homework, which are each labelled as either 'spam' or 'ham'.  In this case, naive Bayes utilizes a Bernoulli model that quantifies the probability of a given word given that the message is either spam or ham.  For example, investigating the text messages here, we find that the word *draw* shows up in spam 27 times, yet in ham only 5.  Thus, we have that
$$ P(X=\mathrm{draw}|Y=\mathrm{ham}) = \frac{5}{5+27}. $$

While this is not particularly strong evidence on its own, we can create a powerful classifier by using the naive assumption in conjunction with all the words in a given message:
$$ P(Y=\mathrm{ham}|\hat{X}) \propto P(Y=\mathrm{ham}) \prod_{i=1}^n P(X=x_i|Y=\mathrm{ham}), $$
$$ P(Y=\mathrm{spam}|\hat{X}) \propto P(Y=\mathrm{spam}) \prod_{i=1}^n P(X=x_i|Y=\mathrm{spam}), $$
where $x_i$ are the words in a given message. 

Your task is to write such a classifier.  I have taken the somewhat tedious step of parsing the data for you, yielding the variables *word_dictionary*, which contains the ham and spam counts for each word, as well as *training_labels*, which provides the spam/ham labels for each text message.  I have also parsed a set of test data: *test_messages* is a list, each entry containing another list of the words in the text message, as well as *test_labels* which contains the spam/ham label for each message.

In [27]:
import numpy as np

# Maps from 'ham' or 'spam' strings to zero or one
def mapper(s):
    if s=='spam':
        return 0
    else:
        return 1

# Read in the text file
f = open('SMSSpamCollection','r')
lines = f.readlines()

# Break out the test data
test_lines = lines[:len(lines)//5]
lines = lines[len(lines)//5:]

# Instantiate the frequency dictionary and an array to
# record whether the line is ham or spam
word_dictionary = {}
training_labels = np.zeros(len(lines),dtype=int)

# Loop over all the training messages
for i,l in enumerate(lines):
    # Split into words
    l = l.lower().split()
    # Record the special first word which always ham or spam
    if l[0]=='ham':
        training_labels[i] = 1
    # For each word in the message, record whether the message was ham or spam
    for w in l[1:]:
        # If we've never seen the word before, add a new dictionary entry
        if w not in word_dictionary:
            word_dictionary[w] = [1,1]
        word_dictionary[w][mapper(l[0])] += 1
     
# Loop over the test messages
test_labels = np.zeros(len(test_lines),dtype=int)
test_messages = []
for i,l in enumerate(test_lines):
    l = l.lower().split()
    if l[0]=='ham':
        test_labels[i] = 1
    test_messages.append(l)


Below, I have provided code skeletons.  Your job is to make the code skeletons into an operational naive Bayes spam detector.  (you may discard these skeletons if you would prefer to code this from scratch).  Note that lines where you will need to change the code are marked with a '#!'.

Your first task is train the model:  

In [31]:
#What is the prior P(Y=ham) ?
ham_prior = .5 #Y is either ham or spam

# What are the class probabilities P(X=word|Y=ham) for each word?
ham_likelihood = {}
for key,val in word_dictionary.items():
    ham_likelihood[key] = word_dictionary[key][1]/(word_dictionary[key][1] + word_dictionary[key][0]) #number of times Y=ham/number of times Y appears 


Your next task is to make predictions on a set of test examples which were held back from the training procedure (see *test_messages* variable).  For each of these messages, compute the ham and spam probabilities.

In [32]:
# Where to hold the ham and spam posteriors
posteriors = np.zeros((len(test_lines),2))

# Loop over all the messages in the test set
for i,m in enumerate(test_messages):
    posterior_ham = 1.0 
    posterior_spam = 1.0 
    
    #! Don't forget to include the prior!
    # Loop over all the words in each message
    for w in m:
        # #! What is the purpose of this try/except handler?
        try:
            posterior_ham *= ham_likelihood[w] #find the product of the probability that each w is in ham
            posterior_spam *= (1 - ham_likelihood[w]) #find the product of the probability that each w is in spam
        except KeyError: #if the word is not in the word dictionary skip it, we have no data on it
            pass
    
    #Since posterior_ham is now the product of all the words in the ith message we must multiply it by ham_prior    
    posterior_ham = ham_prior * posterior_ham 
    #even though ham_prior = spam_prior, if we were to change ham_prior do to more info this would automatically update spam_prior
    posterior_spam = (1-ham_prior) * posterior_spam 
    
    # Notice the normalization factor (denominator) 
    # to turn these into proper probabilities!
    posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
    posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)

Finally, make a ham/spam prediction based on your posterior probabilities.  Compare these to the labels contained in test_labels.  Report the accuracy of your classifier as percentage correct.

In [52]:
test_labels_hat = np.zeros(len(test_lines),dtype=int)
for i,m in enumerate(test_messages):
    if posteriors[i,1] > posteriors[i,0]:
        test_labels_hat[i] = 1

confusionMatrix = np.zeros(shape = (2,2))


for i in range(len(test_labels)):
    k = test_labels[i]
    l = test_labels_hat[i]
    confusionMatrix[k,l] +=1
success = 1 -(confusionMatrix[0,1] + confusionMatrix[1,0])/confusionMatrix.sum()

print(len(test_labels))
print(confusionMatrix)
print(success) 



1114
[[ 105.   63.]
 [   0.  946.]]
0.943447037702


In conclusion, of the 1114 messages in the test_message set. 105 were correctly labeled as spam, 946 were correctly labeled as ham and 63 spams were incorrectly labeled as ham. This is a $94.3\%$ success rate.